# Benchmarking Location Privacy
#### Franklin Harvey

## Question
With regards to privacy I wanted to answer the question "Is it better to truncate the precision of a geospatial point, or to randomly move that point to a nearby location?"

With both approaches we can define how far we would like to move a point. By truncating to 2 decimal places we likely move a point nearly 1km. By fuzzing we can define a radius in which to replace the point, so it will move by at-most that radius.


## Obfuscation
How do we define obfuscation? What is a "good" level of obfuscation and what is a "bad" level.

A good level of obfuscation could loosely be defined as "this location is not the original location". Put from a user's perspective: "This dot that represents me is not at my house".

Our legal department has defined **1km** as a good distance away. So I'll use this definition:

*A good level of obfuscation is for a point to be a kilometer away from its original location*

In [7]:
import random
from math import radians, cos, sin, pi, sqrt, atan2, trunc
import statistics

def h_getFuzzedLocation(latLng, radius, roundDecimals = 5):
    lat = latLng[0]
    lng = latLng[1]
    radiusInDegrees = radius / 111000
    
    u = random.random()
    v = random.random()
    
    w = radiusInDegrees * sqrt(u)
    t = 2 * pi * v
    x = w * cos(t)
    y = w * sin(t)

    new_x = x / cos(radians(lat))

    foundLongitude = round(new_x + lng, roundDecimals)
    foundLatitude = round(y + lat, roundDecimals)
    return [foundLatitude, foundLongitude]

def getFuzzedLocationHundredMeters(latLng):
    return h_getFuzzedLocation(latLng, 100)
    
def getFuzzedLocationFifteenHundredMeters(latLng):
    return h_getFuzzedLocation(latLng, 1500)
    
def getFuzzedLocationTenKilometers(latLng):
    return h_getFuzzedLocation(latLng, 10000)

In [8]:
def getRandomCoordinate(minLat = -180, minLong = -180, maxLat = 180, maxLong = 180):
    lat = random.randrange(minLat, maxLat)
    lat = round(lat + random.random(), 5)
    long = random.randrange(minLong, maxLong)
    long = round(long + random.random(), 5)
    return [lat, long]

def getDistanceInKM(point1, point2):
    R = 6373.0

    lat1 = radians(point1[0])
    lon1 = radians(point1[1])
    lat2 = radians(point2[0])
    lon2 = radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

assert getDistanceInKM([40, -105], [0,0]) == 11282.668188434473

## Fuzzing Once

In [11]:
randomCoords = [getRandomCoordinate() for n in range(100000)]
fuzzedCoords = list(map(getFuzzedLocationFifteenHundredMeters, randomCoords))
distances = []
for i in range(len(fuzzedCoords)):
    distances.append(getDistanceInKM(fuzzedCoords[i], randomCoords[i]))

print("mean distance: " + str(statistics.mean(distances)))
print("Minimum distance (km): " + str(min(distances)))
print("Std. Dev: " + str(statistics.stdev(distances)))

mean distance: 1.0025034488683158
Minimum distance (km): 0.005950625713920481
Std. Dev: 0.3542985728973543


If we fuzz with a radius of 1500 meters we can expect an average distance of **1km** with a standard deviation of **~354m**.

My next question is what is better, to fuzz once with a large radius, or to fuzz many times with a small radius? Keep in mind that as you fuzz many times you start to emulate a [random walk](https://en.wikipedia.org/wiki/Random_walk) algorithm. This is not exactly what we want as it is computationally expensive. We are after final random points, not their journeys.

## Fuzzing many times

In [12]:
randomCoords = [getRandomCoordinate() for n in range(10000)]
numberOfFuzzing = 300
fuzzedCoords = randomCoords.copy()
for i in range(numberOfFuzzing):
    fuzzedCoords = list(map(getFuzzedLocationHundredMeters, fuzzedCoords)).copy()
distances = []
for i in range(len(fuzzedCoords)):
    distances.append(getDistanceInKM(fuzzedCoords[i], randomCoords[i]))

print("mean distance: " + str(statistics.mean(distances)))
print("Minimum distance (km): " + str(min(distances)))
print("Std. Dev: " + str(statistics.stdev(distances)))

mean distance: 1.0902036826233235
Minimum distance (km): 0.008044097713950817
Std. Dev: 0.5713185911711739


By fuzzing by only **100m**, but doing so 300 times to every point, we get a similar level of fuzzing but with a much higher standard deviation. This is only running with 1/10 the number of points and is much slower. I think this is good evidence that fuzzing once by a large radius is a better model for randomizing points than many times with a smaller radius.

## Truncating

In [13]:
def truncatePoint(point, digits = 2) -> float:
    stepper = 10.0 ** digits
    lat = trunc(stepper * point[0]) / stepper
    long = trunc(stepper * point[1]) / stepper
    return [lat, long]

assert truncatePoint([0.1134, 38.4328946792], 2) == [0.11, 38.43]
assert truncatePoint([-0.99, -100.92], 1) == [-0.9, -100.9]

In [14]:
randomCoords = [getRandomCoordinate() for n in range(100000)]
fuzzedCoords = list(map(truncatePoint, randomCoords))

distances = []
for i in range(len(fuzzedCoords)):
    distances.append(getDistanceInKM(fuzzedCoords[i], randomCoords[i]))

print("mean distance: " + str(statistics.mean(distances)))
print("Minimum distance (km): " + str(min(distances)))
print("Std. Dev: " + str(statistics.stdev(distances)))

mean distance: 0.7204556830163453
Minimum distance (km): 0.0012515389226051316
Std. Dev: 0.31587455843953566
